
# Nivel 3, Semana 8: Cifrado Cuántico (BB84)

**Audiencia:** Licenciatura / Avanzado

**Objetivo:** Aplicar los principios de superposición y medición a la criptografía. Derivar formalmente el protocolo BB84, demostrar su seguridad contra ataques simples usando el Teorema de No-Clonación y el intercambio de información-perturbación, y comprender los pasos de post-procesamiento clásico necesarios para obtener una clave segura.

## 1. Puente desde semanas anteriores

* **Semana 1 y 2:** Aprendimos que medir un estado en superposición en la base "incorrecta" da un resultado probabilístico y aleatorio.
* **Semana 5 (Ruido):** Aprendimos que las interacciones con el entorno (como un espía) causan decoherencia y cambian el estado.
* **Semana 7 (VQE):** Aprendimos sobre los bucles de post-procesamiento clásico.

**Semana 8: BB84** utiliza estos principios para resolver el **problema de distribución de claves**:

> *¿Cómo pueden Alice y Bob acordar una clave secreta aleatoria si solo tienen un canal público que Eve podría estar escuchando?*

En física clásica, esto es imposible sin secretos pre-compartidos (como en RSA). En física cuántica, es posible porque **la observación causa perturbación**.

## 2. Formalismo BB84: Dos bases

BB84 es el primer protocolo de distribución de claves cuánticas (QKD), desarrollado por Charles Bennett y Gilles Brassard en 1984. Permite que dos partes, llamadas Alice y Bob, intercambien de forma segura una clave criptográfica inmune a espionaje, gracias a los principios fundamentales de la mecánica cuántica. El protocolo BB84 se basa en dos bases no ortogonales para un solo qubit.

### Las bases

1. **Base rectilínea (Z):**
   * Estados: $|0\rangle$ y $|1\rangle$
   * Observable de medición: $Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$

2. **Base diagonal (X):**
   * Estados: $|+\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}$ y $|-\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}$
   * Observable de medición: $X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$

### Propiedades matemáticas

La propiedad crucial es que estas bases son **mutuamente imparciales**. Si preparas un estado en una base y lo mides en la otra, el resultado es perfectamente aleatorio.

* **Preparar Z, medir X:**
    $$ |\langle + | 0 \rangle|^2 = \left| \frac{1}{\sqrt{2}} \right|^2 = \frac{1}{2} $$
    $$ |\langle - | 0 \rangle|^2 = \frac{1}{2} $$
    (Lo mismo para $|1\rangle$).

* **Preparar X, medir Z:**
    $$ |\langle 0 | + \rangle|^2 = \frac{1}{2} $$
    $$ |\langle 1 | + \rangle|^2 = \frac{1}{2} $$

Esta incertidumbre es el escudo que protege la clave.

## 3. Pasos del protocolo

1. **Transmisión de Alice:**
   Alice genera dos cadenas aleatorias de bits clásicos:
   * **Bits de datos ($a$):** Ejemplo: `1 0 1 1 ...`
   * **Bits de base ($b$):** Ejemplo: `Z X Z X ...` (donde 0=Z, 1=X)

   Ella codifica los qubits según estas elecciones:
   * Si $b_i = Z$ y $a_i = 0 \rightarrow$ Envía $|0\rangle$
   * Si $b_i = Z$ y $a_i = 1 \rightarrow$ Envía $|1\rangle$
   * Si $b_i = X$ y $a_i = 0 \rightarrow$ Envía $|+\rangle$
   * Si $b_i = X$ y $a_i = 1 \rightarrow$ Envía $|-\rangle$

2. **Medición de Bob:**
   Bob **no conoce** las bases de Alice. Él genera su propia cadena aleatoria de **bits de base ($b'$):** Ejemplo: `Z Z X X ...`.
   Mide los qubits entrantes en la base especificada por $b'$.

3. **Sifting (El apretón de manos clásico):**
   * Alice y Bob se comunican por un canal clásico público. Comparan sus cadenas de bases ($b$ y $b'$).
   * **Conservan** los bits donde sus bases coincidieron ($b_i = b'_i$) y **descartan** los bits donde sus bases difirieron ($b_i \neq b'_i$).

   Esto les deja una **clave cruda**. En ausencia de ruido/Eve, esta clave sería idéntica.

#### Ejemplo de Sifting: Generando la clave cruda

Veamos cómo Alice y Bob filtran sus datos para producir una clave cruda compartida.

**1. Fase de transmisión (cuántica)**
* **Alice** genera bits de datos y bases aleatorias. Envía qubits a Bob.
* **Bob** elige bases aleatorias para medir los qubits entrantes.

| Índice | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| :--- | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| **Bit de Alice** | 1 | 0 | 1 | 1 | 0 | 1 | 0 |
| **Base de Alice** | Z | X | Z | X | X | Z | Z |
| **Base de Bob** | Z | Z | X | X | Z | Z | X |
| **Medición de Bob** | 1 | 0 | 0 | 1 | 1 | 1 | 1 |

* *Nota:* En el bit 2, Alice usó X y envió $|+\rangle$ (para 0). Bob usó Z. Midió 0 por azar (50%).
* *Nota:* En el bit 3, Alice usó Z y envió $|1\rangle$. Bob usó X. Midió 0 ($|+\rangle$) por azar.

**2. Fase de Sifting (comunicación clásica)**
Alice y Bob anuncian **solo** sus bases.

* **Alice dice:** "Mis bases fueron: Z, X, Z, X, X, Z, Z"
* **Bob dice:** "Mis bases fueron: Z, Z, X, X, Z, Z, X"

Comparan las listas y encuentran las coincidencias:

| Índice | 1 | 2 | 3 | 4 | 5 | 6 | 7 |
| :--- | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| **¿Coincidencia?** | **SÍ** | NO | NO | **SÍ** | NO | **SÍ** | NO |

**3. Generando la clave cruda**
Conservan los bits 1, 4 y 6, y descartan los bits 2, 3, 5 y 7.

* **Clave cruda de Alice:** `1 1 1` (Índices 1, 4, 6)
* **Clave cruda de Bob:** `1 1 1` (Índices 1, 4, 6)

En este ejemplo ideal sin ruido ni Eve, sus claves coinciden perfectamente. Si Eve estuviera escuchando, algunos de estos bits coincidentes podrían estar cambiados (errores), lo que detectarían en la siguiente fase (verificación de errores).

## 4. Prueba de seguridad: Perturbación de Eve

¿Por qué Eve no puede interceptar el qubit, copiarlo y enviarlo a Bob? Esto se debe al Teorema de No-Clonación en Información Cuántica.

### Teorema de No-Clonación

Eve no puede crear una copia perfecta del qubit $|\psi\rangle$ para almacenarlo.

$$ U(|\psi\rangle \otimes |0\rangle_E) \neq |\psi\rangle \otimes |\psi\rangle $$

Debe medirlo *en ese momento* para obtener información.

### Ataque de interceptar y reenviar

Supongamos que Eve intercepta el qubit, lo mide y envía uno nuevo a Bob según su resultado.

#### Ejemplo de escenario: Eve intercepta un solo qubit

Veamos exactamente qué sucede cuando Eve ataca un bit.

1. **Acción de Alice:**
   * Alice elige **bit de datos `0`** y **base `Z`**.
   * Envía el estado $|\psi\rangle = |0\rangle$.

2. **Intercepción de Eve:**
   * Eve intercepta el qubit. No conoce la base, así que elige al azar medir en la **base X (diagonal)**.
   * Como $|0\rangle = \frac{1}{\sqrt{2}}(|+\rangle + |-\rangle)$, Eve mide $|+\rangle$ o $|-\rangle$ con 50% de probabilidad cada uno.
   * Supongamos que Eve mide $|+\rangle$. Envía un nuevo qubit en estado $|+\rangle$ a Bob.
   * **Daño hecho:** El estado colapsó de $|0\rangle$ a $|+\rangle$. Se perdió información.

3. **Medición de Bob:**
   * Bob elige al azar la **base Z (rectilínea)** (coincidiendo con Alice).
   * Recibe el qubit de Eve, que está en estado $|+\rangle$.
   * Mide $|+\rangle$ en la base Z. Como $|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$, Bob obtiene:
     * **Resultado `0`** con 50% de probabilidad (¡Suerte! Sin error).
     * **Resultado `1`** con 50% de probabilidad (**Error!** Alice envió 0, Bob obtuvo 1).

4. **Sifting y verificación:**
   * Alice y Bob anuncian sus bases: "¡Ambos usamos Z!"
   * Conservan este bit.
   * Luego, comparan una muestra de su clave. Encuentran una discrepancia (Alice tiene 0, Bob tiene 1).
   * **Conclusión:** Como usaron la misma base pero obtuvieron resultados diferentes, saben que **Eve interceptó el qubit.**

* **Caso 1: Eve adivina la base correcta (50% de probabilidad).**
   * Alice envía $|0\rangle$ (base Z).
   * Eve mide en Z. Obtiene `0`. Envía $|0\rangle$ a Bob.
   * Bob mide en Z. Obtiene `0`.
   * **Resultado:** No se introduce error. Eve conoce el bit.

* **Caso 2: Eve adivina la base incorrecta (50% de probabilidad).**
   * Alice envía $|0\rangle$ (base Z).
   * Eve mide en X. Obtiene $|+\rangle$ (aleatorio). Envía $|+\rangle$ a Bob.
   * Bob mide en Z (la base correcta).
     * Como recibe $|+\rangle$, medirá `0` con 50% de probabilidad o `1` con 50% de probabilidad.
   * **Resultado:** Eve ha aleatorizado el estado. Aunque Bob midió en la base correcta, tiene **50% de probabilidad de obtener la respuesta incorrecta (1 en vez de 0)**.

### Cálculo de la tasa de error

Si Alice y Bob comparan una parte de su clave cribada, pueden estimar la **Tasa de Error de Bit Cuántico (QBER)**.

* Eve ataca $N$ bits.
* Adivina mal en $N/2$ bits (50% de probabilidad).
* En esos casos, Bob obtiene un error el 50% de las veces.
* Tasa total de error inducida por Eve:
    $$ E = \frac{1}{2}_{(\text{Eve adivina mal})} \times \frac{1}{2}_{(\text{Bob obtiene error})} = 25\% $$

Si Alice y Bob detectan una tasa de error $\approx 25\%$, saben que Eve está escuchando. En la práctica, cualquier tasa de error mayor a $\approx 11\%$ se considera insegura.

## 5. Post-procesamiento clásico

Encontrar una clave cruda no es suficiente. El hardware real tiene ruido (genera errores sin Eve), y Eve podría tener información parcial (menos del 25% de error).

### Reconciliación de errores (Information Reconciliation)

Alice y Bob deben corregir los errores en sus claves sin revelar la clave misma.

#### Ejemplo: Protocolo Cascade (paso a paso)

Así encuentran y corrigen errores de forma interactiva:

1. **Permutación (mezclar):**
   * Alice y Bob acuerdan una permutación aleatoria para mezclar los bits de la clave y dispersar cualquier error "en ráfaga" por toda la clave.

2. **División en bloques e intercambio de paridad:**
   * Dividen sus claves en bloques de tamaño fijo (por ejemplo, $k$ bits).
   * Calculan la **paridad** (suma módulo 2) de cada bloque.
   * Alice envía sus paridades a Bob.

3. **Comparación de paridades:**
   * **Coincidencia:** Si las paridades coinciden (por ejemplo, ambas 0), asumen que no hay errores (o hay un número par de errores) y continúan.
   * **Desajuste:** Si las paridades difieren, saben que hay un **número impar de errores** (usualmente 1) en ese bloque.

4. **Búsqueda binaria (localizar el error):**
   * Dividen el bloque en mitades izquierda y derecha.
   * Comparan la paridad de la mitad izquierda.
   * Si hay desajuste, el error está en la izquierda. Si coincide, está en la derecha.
   * Repiten hasta aislar el bit erróneo.
   * Bob corrige ese bit.

5. **Iteración:**
   * Vuelven a mezclar toda la clave con una *nueva* permutación y repiten el proceso para atrapar errores pares restantes.

### 5.B. Amplificación de privacidad

Incluso después de corregir errores, Eve podría conocer, digamos, el 10% de los bits. Alice y Bob quieren una clave más corta que Eve no conozca en absoluto.

* **Método:** Hashing universal.
* Acuerdan una función hash aleatoria $f$ (como XOR de subconjuntos aleatorios de bits).
* Nueva clave $K_{final} = f(K_{cruda})$.
* **Resultado:** Si Eve conocía 10 bits de una clave de 100 bits, y la comprimimos a 50 bits, el conocimiento de Eve sobre los nuevos 50 bits cae exponencialmente a casi cero.

**Fórmula final de la tasa de clave segura:**
$$ R \approx 1 - H_1(e) - H_2(e) $$
Donde $H(e)$ es la entropía binaria de la tasa de error.
* 1er $H_1(e)$: Costo de corrección de errores.
* 2do $H_2(e)$: Bits perdidos por amplificación de privacidad (asumiendo que Eve conoce tanto como permite el ruido).
